# DNS (Thermospheric density)

In [ ]:
SERVER_URL = "https://disc.vires.services/ows"

In [ ]:
%load_ext watermark
%watermark -i -v -p viresclient,pandas,xarray,matplotlib,cartopy

In [ ]:
import datetime as dt

import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from viresclient import SwarmRequest

## Product information

Thermospheric density products are available through the following collections, both from Swarm, and from other spacecraft (via the [TOLEOS project](https://earth.esa.int/eogateway/activities/toleos))

- Thermospheric density derived from Swarm A, B, C (using the accelerometer):  
  [`SW_OPER_DNSxACC_2_`](https://swarmhandbook.earth.esa.int/catalogue/SW_DNSxACC_2_)
- Thermospheric density derived from Swarm A, B, C (using only the orbit determination):  
  [`SW_OPER_DNSxPOD_2_`](https://swarmhandbook.earth.esa.int/catalogue/SW_DNSxPOD_2_)
- CHAMP:  
  [`CH_OPER_DNS_ACC_2_`](https://swarmhandbook.earth.esa.int/catalogue/CH_DNS_ACC_2_)
- GRACE 1 & 2:  
  [`GR_OPER_DNSxACC_2_`](https://swarmhandbook.earth.esa.int/catalogue/GR_DNSxACC_2_)
- GRACE-FO 1:  
  [`GF_OPER_DNSxACC_2_`](https://swarmhandbook.earth.esa.int/catalogue/GF_DNSxACC_2_)

We can check the available parameter names with:

In [ ]:
request = SwarmRequest(SERVER_URL)
for collection in ("SW_OPER_DNSAACC_2_", "SW_OPER_DNSAPOD_2_", "CH_OPER_DNS_ACC_2_", "GR_OPER_DNS1ACC_2_", "GF_OPER_DNS1ACC_2_"):
    print(f"{collection}:\n{request.available_measurements(collection)}\n")

We can check for available time windows with:

In [ ]:
request.available_times("SW_OPER_DNSAPOD_2_")

In [ ]:
request.available_times("GF_OPER_DNS1ACC_2_")

## Fetching neutral density

In [ ]:
request = SwarmRequest(SERVER_URL)
request.set_collection(f"SW_OPER_DNSAACC_2_")
request.set_products(
    measurements=["density"],
)
data = request.get_between(dt.datetime(2015, 1, 1), dt.datetime(2015, 1, 2))
data.as_xarray()

## Fetching from multiple spacecraft

We will fetch the data from around the geomagnetic storm event that affected a SpaceX Starlink launch and subsequent loss of spacecraft. See for example:  
[Starlink Satellite Losses During the February 2022 Geomagnetic Storm Event: Science, Technical and Economic Consequences, Policy, and Mitigation](https://agupubs.onlinelibrary.wiley.com/doi/toc/10.1002/(ISSN)1542-7390.STRLNK2022)

In [ ]:
def fetch_density(mission="SW", spacecraft="A", source="ACC", start_time=None, end_time=None):
    request = SwarmRequest(SERVER_URL)
    request.set_collection(f"{mission}_OPER_DNS{spacecraft}{source}_2_")
    request.set_products(
        measurements=["density"],
        auxiliaries=["QDLat", "OrbitNumber", "OrbitDirection", "QDOrbitDirection"],
    )
    data = request.get_between(start_time, end_time, asynchronous=False, show_progress=False)
    return data.as_xarray()


start_time = dt.datetime(2022, 2, 1)
end_time = dt.datetime(2022, 2, 8)
# ds_SwA = fetch_density(mission="SW", spacecraft="A", source="POD", start_time=start_time, end_time=end_time)
# ds_SwB = fetch_density(mission="SW", spacecraft="B", source="POD", start_time=start_time, end_time=end_time)
ds_SwC = fetch_density(mission="SW", spacecraft="C", source="POD", start_time=start_time, end_time=end_time)
ds_GF1 = fetch_density(mission="GF", spacecraft="1", source="ACC", start_time=start_time, end_time=end_time)

Now we have the data available from several datasets, e.g.:

In [ ]:
# Use Swarm-C dataset, extract one day, and plot density
ds_SwC.sel(Timestamp=slice("2022-02-02", "2022-02-02"))["density"].plot.line(figsize=(10, 2));

You can check which source products have been accessed:

In [ ]:
ds_SwC.attrs["Sources"]

## Visualisation example

Let's try to display the density data as a function of time and latitude...

In [ ]:
ds_GF1.plot.scatter(x="Timestamp", y="Latitude", hue="density", s=2, edgecolors="face", figsize=(10, 4));

Now let's explore that a bit more to have more control over the figure

In [ ]:
def select_north(ds):
    return ds.where(ds["Latitude"] > 0, drop=True)

def select_south(ds):
    return ds.where(ds["Latitude"] < 0, drop=True)

def select_ascending(ds):
    return ds.where(ds["OrbitDirection"] == 1, drop=True)

def select_descending(ds):
    return ds.where(ds["OrbitDirection"] == -1, drop=True)

def get_times_at_orbits(ds, orbitnos: list):
    da = ds["OrbitNumber"]
    t = []
    for orbitno in orbitnos:
        try:
            _t = da.where(da == orbitno, drop=True).isel({"Timestamp": 0})["Timestamp"].values.astype("datetime64[s]").astype(dt.datetime)
        except IndexError:
            _t = None
        t.append(_t)
    return t

def get_orbits_at_times(ds, times: list):
    da = ds["OrbitNumber"]
    return list(da.sel(Timestamp=times, method="nearest").values.astype(int))

In [ ]:
norm_gf =  mpl.colors.Normalize(vmin=0, vmax=7.5e-13)
norm_sw = mpl.colors.Normalize(vmin=0, vmax=20e-13)

def plot_density(ds=ds_GF1, norm=norm_gf, title="GRACE-FO 1"):
    
    cmap = "viridis"
    plot_kwargs = {"norm": norm, "edgecolors": None, "s": 2, "marker": "s", "cmap": cmap}
    
    fig, axes = plt.subplots(2, 1, figsize=(10, 4), sharex=True)
    # Plot the four segments of each orbit in sequence
    # North
    _ds = select_ascending(select_north(ds))
    axes[0].scatter(x=_ds["OrbitNumber"], y=_ds["Latitude"], c=_ds["density"], **plot_kwargs)
    _ds = select_descending(select_north(ds))
    axes[0].scatter(x=_ds["OrbitNumber"]+.5, y=_ds["Latitude"], c=_ds["density"], **plot_kwargs)
    # South
    _ds = select_descending(select_south(ds))
    axes[1].scatter(x=_ds["OrbitNumber"]+0.5, y=_ds["Latitude"], c=_ds["density"], **plot_kwargs)
    _ds = select_ascending(select_south(ds))
    axes[1].scatter(x=_ds["OrbitNumber"]+1.0, y=_ds["Latitude"], c=_ds["density"], **plot_kwargs)
    # Adjust axes
    axes[0].set_ylim(0, 90)
    axes[1].set_ylim(-90, 0)
    fig.subplots_adjust(hspace=0)
    axes[1].set_ylabel("Latitude")
    # axes[0].set_backgr
    fig.suptitle(title)
    # Move xticks to align with day starts
    t_start_end = ds["Timestamp"].isel(Timestamp=[0, -1]).values.astype("datetime64[s]").astype(dt.datetime)
    times_to_mark = pd.date_range(start=t_start_end[0], end=t_start_end[1], freq="D")
    orbits_to_mark = get_orbits_at_times(ds, times_to_mark)
    axes[1].set_xticks(orbits_to_mark)
    # Replace orbit numbers with the times
    times_formatted = [_t.strftime("%Y-%m-%d") for _t in times_to_mark]
    axes[1].set_xticklabels(times_formatted)
    # Add colorbar
    cbar_ax = fig.add_axes([0.95, 0.3, 0.01, 0.4])
    sm = mpl.cm.ScalarMappable(cmap=plot_kwargs["cmap"], norm=plot_kwargs["norm"])
    sm.set_array([])
    cb = fig.colorbar(sm, cax=cbar_ax)
    cb.set_label("density [kg m$^{-3}$]")

In [ ]:
plot_density(ds_GF1, norm_gf, "GRACE-FO 1")

In [ ]:
plot_density(ds_SwC, norm_sw, "Swarm C")